In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/DataScience2019_MRI/Behavioral/AllData.csv', low_memory=False)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103

In [3]:
data

,Anonymized.ID,Subject.Type,Visit,Days.since.enrollment,EID,START_DATE,Patient_ID,Enrollment.Year,Enrollment.Season,Sex,...,YSR_TP,YSR_TP_T,YSR_Ext,YSR_Ext_T,YSR_Int,YSR_Int_T,YSR_OP,YSR_C,YSR_Total,YSR_Total_T
0,A00078864,UNKNOWN,V1,0,NDARYM832PX3,1901-01-01T07:00:00Z,NDARYM832PX3,2015,Spring,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A00078865,UNKNOWN,V1,0,NDARNJ687DMC,1901-01-01T07:00:00Z,NDARNJ687DMC,2015,Spring,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A00078866,UNKNOWN,V1,0,NDARRM363BXZ,1901-01-01T07:00:00Z,NDARRM363BXZ,2015,Spring,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A00078867,UNKNOWN,V1,0,NDARUW586LLL,1901-01-01T07:00:00Z,NDARUW586LLL,2015,Spring,1,...,15.0,75.0,16.0,60.0,29.0,68.0,8.0,40.0,85.0,68.0
4,A00078868,UNKNOWN,V1,0,NDARDC298NW4,1901-01-01T07:00:00Z,NDARDC298NW4,2015,Spring,0,...,9.0,67.0,10.0,52.0,26.0,72.0,11.0,34.0,70.0,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2091,A00093557,UNKNOWN,V1,0,NDARYZ986HEW,1901-01-01T07:00:00Z,NDARYZ986HEW,2018,Summer,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2092,A00093558,UNKNOWN,V1,0,NDARPM572ZZV,1901-01-01T07:00:00Z,NDARPM572ZZV,2018,Summer,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2093,A00093559,UNKNOWN,V1,0,NDARYA955CY1,1901-01-01T07:00:00Z,NDARYA955CY1,2018,Fall,0,...,6.0,60.0,11.0,53.0,13.0,59.0,3.0,23.0,47.0,58.0
2094,A00093560,UNKNOWN,V1,0,NDARRT283RHB,1901-01-01T07:00:00Z,NDARRT283RHB,2018,Fall,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
feat_to_drop = data.nunique() == 1
dropping_features = list(data.columns[feat_to_drop])

print('features to drop:', len(dropping_features))

features to drop: 289


In [18]:
data.drop(columns=dropping_features, inplace=True)

In [20]:
print('Total len of data', len(data))

data_nulls = data.isnull().apply(sum, axis=0).sort_values(ascending=False)

Total len of data 2096


In [27]:
# drop values for which more than threshold percent are NaNs
threshold = 0.8

dropped_columns = data_nulls.values > threshold * len(data)

In [29]:
import numpy as np

np.sum(dropped_columns)

2753

In [32]:
data.drop(columns=data_nulls.index[dropped_columns], inplace=True)

In [45]:
all_dropped_features = list(data_nulls.index[dropped_columns].values) + dropping_features

len(all_dropped_features)

3042

In [33]:
data.columns

Index(['Anonymized.ID', 'EID', 'Patient_ID', 'Enrollment.Year',
       'Enrollment.Season', 'Sex', 'Age', 'Study.Site', 'Commercial.Use',
       'Release.Number',
       ...
       'YSR_TP', 'YSR_TP_T', 'YSR_Ext', 'YSR_Ext_T', 'YSR_Int', 'YSR_Int_T',
       'YSR_OP', 'YSR_C', 'YSR_Total', 'YSR_Total_T'],
      dtype='object', length=4000)

## Column observations

TODO: maybe theses observations may not be done on the initial dataset but on the reduced one based on the number of patients that actually have a diagnosis

In [53]:
whole_data = pd.read_csv('data/DataScience2019_MRI/Behavioral/AllData.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103

#### Diagnoses

In [253]:
disorders = set()

for i in range(1, 11):
    disorders.update(whole_data['DX_' + str(i).zfill(2) + '_Cat'].unique())
    
disorders = list(disorders)
disorders.remove(' ')
disorders = [x for x in disorders if str(x) != 'nan']

counter = np.zeros(len(disorders), dtype=np.int64)

for i in range(1, 11):
    test = whole_data['DX_' + str(i).zfill(2) + '_Cat'].values
    
    for j, disorder in enumerate(disorders):
        counter[j] += np.sum(test == disorder)

for j, disorder in enumerate(disorders):
    print('{0: <60}: {1}'.format(disorder, counter[j]))

Other Conditions That May Be a Focus of Clinical Attention  : 23
Neurodevelopmental Disorders                                : 2081
No Diagnosis Given                                          : 235
Schizophrenia Spectrum and other Psychotic Disorders        : 10
Gender Dysphoria                                            : 1
Disruptive, Impulse Control and Conduct Disorders           : 35
Feeding and Eating Disorders                                : 10
Anxiety Disorders                                           : 597
Trauma and Stressor Related Disorders                       : 77
Sleep-Wake Disorders                                        : 3
No Diagnosis Given: Incomplete Eval                         : 278
Elimination Disorders                                       : 96
Substance Related and Addictive Disorders                   : 9
Apppendix (Neurobehavioral Disorder)                        : 1
Obsessive Compulsive and Related Disorders                  : 74
Neurocognitive Disorders

In [258]:
disorders = set()

for i in range(1, 11):
    disorders.update(whole_data['DX_' + str(i).zfill(2) + '_Sub'].unique())
    
disorders = list(disorders)
disorders = [x for x in disorders if str(x) != 'nan']

counter = np.zeros(len(disorders), dtype=np.int64)

for i in range(1, 11):
    test = whole_data['DX_' + str(i).zfill(2) + '_Sub'].values
    
    for j, disorder in enumerate(disorders):
        counter[j] += np.sum(test == disorder)

for j, disorder in enumerate(disorders):
    print('{0: <60}: {1}'.format(disorder, counter[j]))

Attention-Deficit/Hyperactivity Disorder                    : 1022
Motor Disorder                                              : 99
Abuse and Neglect: Child Maltreatment and Neglect Problems: Child Neglect: 1
Tobacco-Related                                             : 2
Autism Spectrum Disorder                                    : 308
Intellectual Disability                                     : 57
Acute                                                       : 1
Past                                                        : 1
Nonadherence to Medical Treatment                           : 16
Alcohol-Related                                             : 2
Relational Problems: Problems Related to Family Upbringing  : 5
Cannabis-Related                                            : 5
Specific Learning Disorder                                  : 366
Communication Disorder                                      : 229
Other Neurodevelopmental Disorders                          : 3
Problems Relate

In [109]:
# For some cases ACE_Score is not NaN although all scores ACE_{01-10} are NaN
# Also should we care about all the ACE random columns?

positions = list(set(whole_data[~ np.isnan(whole_data['ACE_Score'])].index) - \
                    set(whole_data[~ np.isnan(whole_data['ACE_01'])].index))

print(positions)

relevant_columns = ['ACE_0' + str(i) for i in range(1, 10)] + ['ACE_10', 'ACE_Score']
whole_data[relevant_columns].iloc[positions]

[2072, 1996, 1956]


,ACE_01,ACE_02,ACE_03,ACE_04,ACE_05,ACE_06,ACE_07,ACE_08,ACE_09,ACE_10,ACE_Score
2072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [122]:
# Percentage of NaN values for APQ_P
# for all columns the same values are Nan

cols = ['APQ_P_' + i for i in ['INV', 'PP', 'PM', 'ID', 'CP', 'Total', 'OPD']]
print('For the columns:', cols)

print('This percentage of values are Nan:', np.sum(np.isnan(whole_data[cols[0]].values)) / len(whole_data))

For the columns: ['APQ_P_INV', 'APQ_P_PP', 'APQ_P_PM', 'APQ_P_ID', 'APQ_P_CP', 'APQ_P_Total', 'APQ_P_OPD']
This percentage of values are Nan: 0.15314885496183206


In [127]:
# Percentage of NaN values for APQ_SR
# for all columns the same values are Nan

cols = ['APQ_SR_' + i for i in ['INV_D', 'INV_M', 'PP', 'PM', 'CP', 'OPD', 'Total']]
print('For the columns:', cols)

print('This percentage of values are Nan:', np.sum(np.isnan(whole_data[cols[0]].values)) / len(whole_data))

For the columns: ['APQ_SR_INV_D', 'APQ_SR_INV_M', 'APQ_SR_PP', 'APQ_SR_PM', 'APQ_SR_CP', 'APQ_SR_OPD', 'APQ_SR_Total']
This percentage of values are Nan: 0.16173664122137404


In [136]:
# Percentage of NaN values for ARI_P

cols = ['ARI_P_0' + str(i) for i in range(1, 8)] + ['ARI_P_Total_Score']
print('For the columns:', cols)

for col in cols:
    print('This percentage of values are Nan:', np.sum(np.isnan(whole_data[col].values)) / len(whole_data))

For the columns: ['ARI_P_01', 'ARI_P_02', 'ARI_P_03', 'ARI_P_04', 'ARI_P_05', 'ARI_P_06', 'ARI_P_07', 'ARI_P_Total_Score']
This percentage of values are Nan: 0.1340648854961832
This percentage of values are Nan: 0.13454198473282442
This percentage of values are Nan: 0.1340648854961832
This percentage of values are Nan: 0.13454198473282442
This percentage of values are Nan: 0.1340648854961832
This percentage of values are Nan: 0.13549618320610687
This percentage of values are Nan: 0.13454198473282442
This percentage of values are Nan: 0.1340648854961832


In [137]:
# Percentage of NaN values for ARI_S

cols = ['ARI_S_0' + str(i) for i in range(1, 8)] + ['ARI_P_Total_Score']
print('For the columns:', cols)

for col in cols:
    print('This percentage of values are Nan:', np.sum(np.isnan(whole_data[col].values)) / len(whole_data))

For the columns: ['ARI_S_01', 'ARI_S_02', 'ARI_S_03', 'ARI_S_04', 'ARI_S_05', 'ARI_S_06', 'ARI_S_07', 'ARI_P_Total_Score']
This percentage of values are Nan: 0.11259541984732824
This percentage of values are Nan: 0.11307251908396947
This percentage of values are Nan: 0.11402671755725191
This percentage of values are Nan: 0.11259541984732824
This percentage of values are Nan: 0.11307251908396947
This percentage of values are Nan: 0.11307251908396947
This percentage of values are Nan: 0.11354961832061068
This percentage of values are Nan: 0.1340648854961832


## test

Attention-Deficit/Hyperactivity Disorder                    : 1022
Motor Disorder                                              : 99
Abuse and Neglect: Child Maltreatment and Neglect Problems: Child Neglect: 1
Tobacco-Related                                             : 2
Autism Spectrum Disorder                                    : 308
Intellectual Disability                                     : 57
Acute                                                       : 1
Past                                                        : 1
Nonadherence to Medical Treatment                           : 16
Alcohol-Related                                             : 2
Relational Problems: Problems Related to Family Upbringing  : 5
Cannabis-Related                                            : 5
Specific Learning Disorder                                  : 366
Communication Disorder                                      : 229
Other Neurodevelopmental Disorders                          : 3
Problems Relate

## Simple algorithm

In [192]:
data = pd.read_csv('data/DataScience2019_MRI/Behavioral/AllData.csv', low_memory=False)

# For start only keep patients for which we have actual diagnosis
# not having a diagnosis means that they are healthy ?
data = data[(data['NoDX'] == 'Yes') | (data['NoDX'] == 'No')]

important_columns = ['Sex', 'Age', 'BMI', 'Handedness', 'WISC_FSIQ']

data = data[important_columns]

# relevant_columns = []

# other_columns = ['Participant_Status']

In [180]:
data['Handedness'].fillna(0, inplace=True)

In [181]:
data['BMI'].fillna((data['BMI'].mean()), inplace=True)

In [182]:
data['WISC_FSIQ'].fillna((data['WISC_FSIQ'].mean()), inplace=True)

In [185]:
for col in whole_data.columns:
    if 'DX' in col:
        print(col)

NoDX
DX_01_Cat
DX_01_Sub
DX_01
DX_01_Spec
DX_01_Code
DX_01_Time
DX_01_Confirmed
DX_01_Presum
DX_01_RC
DX_01_RuleOut
DX_01_ByHx
DX_01_New
DX_01_Rem
DX_01_PRem
DX_01_Past_Doc
DX_02_Cat
DX_02_Sub
DX_02
DX_02_Spec
DX_02_Code
DX_02_Time
DX_02_Confirmed
DX_02_Presum
DX_02_RC
DX_02_RuleOut
DX_02_ByHx
DX_02_New
DX_02_Rem
DX_02_PRem
DX_02_Past_Doc
DX_03_Cat
DX_03_Sub
DX_03
DX_03_Spec
DX_03_Code
DX_03_Time
DX_03_Confirmed
DX_03_Presum
DX_03_RC
DX_03_RuleOut
DX_03_ByHx
DX_03_New
DX_03_Rem
DX_03_PRem
DX_03_Past_Doc
DX_04_Cat
DX_04_Sub
DX_04
DX_04_Spec
DX_04_Code
DX_04_Time
DX_04_Confirmed
DX_04_Presum
DX_04_RC
DX_04_RuleOut
DX_04_ByHx
DX_04_New
DX_04_Rem
DX_04_PRem
DX_04_Past_Doc
DX_05_Cat
DX_05_Sub
DX_05
DX_05_Spec
DX_05_Code
DX_05_Time
DX_05_Confirmed
DX_05_Presum
DX_05_RC
DX_05_RuleOut
DX_05_ByHx
DX_05_New
DX_05_Rem
DX_05_PRem
DX_05_Past_Doc
DX_06_Cat
DX_06_Sub
DX_06
DX_06_Spec
DX_06_Code
DX_06_Time
DX_06_Confirmed
DX_06_Presum
DX_06_RC
DX_06_RuleOut
DX_06_ByHx
DX_06_New
DX_06_Rem
DX_06_PRem
DX